# Training XGBoost

## Code for training a XGBoost model using cross-validation

In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier 
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_score, \
                            confusion_matrix, accuracy_score, f1_score,recall_score
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold, cross_val_score, RepeatedKFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""Functions def"""
def read_data(raw_clinical_note):
    data = pd.read_csv(raw_clinical_note, header=0,na_filter=True)
    return data

def saving_model(model,model_name,logs_file):
    model_name = model_name + '.pkl'
    url_save = os.path.join(logs_file,str(model_name))
    joblib.dump(model, url_save)

In [3]:
"""Loading training sets"""
path = "C:/Users/Salvador/Modelo_COVID19/Libretas manuscrito/BCM Infectius diseases/BCM/"
x_train = pd.read_csv(path + "X_train.csv")
y_train = pd.read_csv(path + "y_train.csv")
print ("x_train:", x_train.shape, "y_train:",y_train.shape)

x_train: (9251, 35) y_train: (9251, 2)


In [4]:
x_train = x_train.drop(['Unnamed: 0'],axis=1)
y_train = y_train.drop(['Unnamed: 0'],axis=1)
print ("x_train:", x_train.shape, "y_train:", y_train.shape)

x_train: (9251, 34) y_train: (9251, 1)


#y_train = np.array(y_train)

In [5]:
y_train.value_counts()

lethality
0            7953
1            1298
dtype: int64

In [6]:
x_train

,Sex,Age,Fever,Cough,Odynophagia,Dyspnea,Irritability,Diarrhea,Chest pain,Chills,...,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular disease,Obesity,CKD,Smoking,Use of antipyretics,Days elapsed
0,0.923870,-1.200365,0.426006,0.390861,-1.264644,-0.886320,-0.425121,-0.519490,-0.696870,0.957761,...,-0.142059,-0.138453,-0.078738,-0.560746,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,0.226939
1,-1.082403,0.340091,0.426006,0.390861,0.790736,-0.886320,-0.425121,-0.519490,-0.696870,0.957761,...,-0.142059,-0.138453,-0.078738,-0.560746,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,-1.026691
2,-1.082403,0.054821,-2.347382,0.390861,-1.264644,1.128261,-0.425121,-0.519490,-0.696870,0.957761,...,7.039312,-0.138453,-0.078738,1.783339,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,-0.190938
3,0.923870,-1.314472,-2.347382,0.390861,0.790736,-0.886320,-0.425121,-0.519490,-0.696870,0.957761,...,-0.142059,-0.138453,-0.078738,-0.560746,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,1.898446
4,-1.082403,-0.173394,-2.347382,0.390861,0.790736,-0.886320,-0.425121,-0.519490,1.434989,0.957761,...,-0.142059,-0.138453,-0.078738,-0.560746,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,-1.026691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9246,-1.082403,-0.515718,0.426006,0.390861,-1.264644,-0.886320,-0.425121,1.924966,1.434989,-1.044102,...,-0.142059,-0.138453,-0.078738,-0.560746,-0.147124,-0.452107,-0.172061,-0.258154,0.949495,-0.608815
9247,-1.082403,0.682414,0.426006,0.390861,0.790736,1.128261,-0.425121,1.924966,-0.696870,0.957761,...,-0.142059,-0.138453,-0.078738,1.783339,-0.147124,2.211867,-0.172061,-0.258154,-1.053191,0.226939
9248,0.923870,1.538223,0.426006,0.390861,-1.264644,1.128261,2.352274,-0.519490,-0.696870,-1.044102,...,-0.142059,-0.138453,-0.078738,1.783339,-0.147124,-0.452107,-0.172061,3.873653,-1.053191,-0.608815
9249,0.923870,0.796522,0.426006,0.390861,-1.264644,-0.886320,2.352274,-0.519490,1.434989,0.957761,...,-0.142059,-0.138453,-0.078738,1.783339,-0.147124,-0.452107,-0.172061,-0.258154,-1.053191,0.644816


In [7]:
print(y_train.value_counts())
y_train = np.array(y_train)

lethality
0            7953
1            1298
dtype: int64


In [8]:
estimator = xgb.XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=442)
sample_weights = class_weight.compute_sample_weight('balanced', y_train)
ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
parameters={"n_estimators":[200, 250, 300], 
            "learning_rate": [0.03, 0.05, 0.1],
            "objective": ['binary:logistic'], 
            "max_depth":[5, 6],
            "seed": [422], 
            "eval_metric": ['aucpr'],
            "reg_alpha" : [0.03],
            "rate_drop": [0.3, 0.5],
            "subsample" : [0.3],
            "gamma":[2],
            "scale_pos_weight": [ratio],}

grid = GridSearchCV(estimator=estimator, param_grid=parameters,cv = cv, scoring='roc_auc',refit = True)
grid.fit(x_train,y_train,sample_weight = sample_weights)

[09:14:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:14:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:14:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[09:15:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:15:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:15:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[09:15:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:15:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:15:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[09:16:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:16:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:16:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[09:16:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:16:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:16:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

...

[10:34:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[10:35:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[10:36:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:36:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:36:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

[10:36:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:36:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:36:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bi

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=442),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=N...
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
             param_grid={'eval_metric': ['aucpr'], 'gamma': [2],
                         'learning_rate': [0.03, 0.05, 0.1],
                         'max_depth': [5, 6], 'n_estimators': [200, 250, 300],
                         'objective': ['binary:logistic'],
                         'rate_drop': [0.3, 0.5], 'reg_alpha': [0.03],
                         'scale_pos_weight': [6.127118644067797], 'seed': [422],
                         'subsample': [0.3]},
             scoring='roc_auc')

print('Best params achieved via GridSearch: \n',grid.best_params_)
print('Best score in training:', grid.best_score_)

In [9]:
print('Best params achieved via GridSearch: \n',grid.best_params_)
print('Best score in training:', grid.best_score_)

Best params achieved via GridSearch: 
 {'eval_metric': 'aucpr', 'gamma': 2, 'learning_rate': 0.03, 'max_depth': 5, 'n_estimators': 200, 'objective': 'binary:logistic', 'rate_drop': 0.3, 'reg_alpha': 0.03, 'scale_pos_weight': 6.127118644067797, 'seed': 422, 'subsample': 0.3}
Best score in training: 0.8483270765965153


In [10]:
"""Visualing metrics from gridSearchCV"""
cv_results = pd.DataFrame.from_dict(grid.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eval_metric,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,param_objective,...,split93_test_score,split94_test_score,split95_test_score,split96_test_score,split97_test_score,split98_test_score,split99_test_score,mean_test_score,std_test_score,rank_test_score
0,1.313699,0.221499,0.009945,0.003829,aucpr,2,0.03,5,200,binary:logistic,...,0.847107,0.855965,0.826444,0.808699,0.838587,0.871524,0.859478,0.848327,0.015182,1
1,1.461751,0.163200,0.010550,0.002151,aucpr,2,0.03,5,200,binary:logistic,...,0.847107,0.855965,0.826444,0.808699,0.838587,0.871524,0.859478,0.848327,0.015182,1
2,1.870775,0.223601,0.011421,0.002906,aucpr,2,0.03,5,250,binary:logistic,...,0.846575,0.854427,0.827247,0.808892,0.838258,0.873072,0.860406,0.847965,0.015241,3
3,1.778907,0.279511,0.011176,0.003586,aucpr,2,0.03,5,250,binary:logistic,...,0.846575,0.854427,0.827247,0.808892,0.838258,0.873072,0.860406,0.847965,0.015241,3
4,1.710741,0.365582,0.010206,0.007693,aucpr,2,0.03,5,300,binary:logistic,...,0.846604,0.854814,0.825776,0.808292,0.837339,0.873498,0.858945,0.847275,0.015389,5
5,1.358955,0.169714,0.005830,0.001277,aucpr,2,0.03,5,300,binary:logistic,...,0.846604,0.854814,0.825776,0.808292,0.837339,0.873498,0.858945,0.847275,0.015389,5
6,1.038376,0.114949,0.005514,0.001142,aucpr,2,0.03,6,200,binary:logistic,...,0.841055,0.855423,0.823222,0.809531,0.837755,0.874040,0.858452,0.846114,0.015237,7
7,1.081391,0.137648,0.006137,0.004331,aucpr,2,0.03,6,200,binary:logistic,...,0.841055,0.855423,0.823222,0.809531,0.837755,0.874040,0.858452,0.846114,0.015237,7
8,1.344008,0.155124,0.005769,0.000942,aucpr,2,0.03,6,250,binary:logistic,...,0.839690,0.852743,0.823880,0.808089,0.836507,0.874233,0.857746,0.845467,0.015764,11
9,1.348503,0.187354,0.005860,0.001314,aucpr,2,0.03,6,250,binary:logistic,...,0.839690,0.852743,0.823880,0.808089,0.836507,0.874233,0.857746,0.845467,0.015764,11


In [11]:
"""Saving AUC (cross-validation)"""
cv_results = pd.DataFrame.from_dict(grid.cv_results_)
auc_training = cv_results.mean_test_score
auc_training.to_csv(path+"AUC_crossval_XGBoost_0.csv")

In [12]:
""" Re-training the model with the best params"""
best_params = grid.best_params_
model = xgb.XGBClassifier(**best_params)
model = model.fit(x_train,y_train)

[10:37:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [13]:
"""Saving model"""
saving_model(model,"XGBoost_model_0",path)